In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 3
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.25 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_1S
            epoch = 120
            lr = 0.00032
        elif(TL==1):
            trainData = data_3S
            epoch = 120
            lr = 0.00032
        else:
            trainData = data_5S
            epoch = 120
            lr = 0.00032
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/120 Train_Loss: 1.8389647006988525 Train_acc: 30.075001%
Train Epoch: 2/120 Train_Loss: 1.1935631036758423 Train_acc: 56.800003%
Train Epoch: 3/120 Train_Loss: 1.241291880607605 Train_acc: 70.962502%
Train Epoch: 4/120 Train_Loss: 0.8501532077789307 Train_acc: 77.987503%
Train Epoch: 5/120 Train_Loss: 0.7979494333267212 Train_acc: 82.437508%
Train Epoch: 6/120 Train_Loss: 0.5806000232696533 Train_acc: 84.975006%
Train Epoch: 7/120 Train_Loss: 0.4122045338153839 Train_acc: 87.437508%
Train Epoch: 8/120 Train_Loss: 0.6280247569084167 Train_acc: 88.037506%
Train Epoch: 9/120 Train_Loss: 0.5212181210517883 Train_acc: 88.300003%
Train Epoch: 10/120 Train_Loss: 0.294162780046463 Train_acc: 89.237503%
Train Epoch: 11/120 Train

Train Epoch: 109/120 Train_Loss: 0.3159944415092468 Train_acc: 93.212502%
Train Epoch: 110/120 Train_Loss: 0.19029465317726135 Train_acc: 93.225006%
Train Epoch: 111/120 Train_Loss: 0.17220014333724976 Train_acc: 93.212502%
Train Epoch: 112/120 Train_Loss: 0.007337426301091909 Train_acc: 93.212502%
Train Epoch: 113/120 Train_Loss: 0.21491704881191254 Train_acc: 93.237503%
Train Epoch: 114/120 Train_Loss: 0.04988013952970505 Train_acc: 93.225006%
Train Epoch: 115/120 Train_Loss: 0.1337362825870514 Train_acc: 93.237503%
Train Epoch: 116/120 Train_Loss: 0.27801403403282166 Train_acc: 93.225006%
Train Epoch: 117/120 Train_Loss: 0.12390665709972382 Train_acc: 93.225006%
Train Epoch: 118/120 Train_Loss: 0.08170928806066513 Train_acc: 93.262505%
Train Epoch: 119/120 Train_Loss: 0.276081919670105 Train_acc: 93.175003%
Train Epoch: 120/120 Train_Loss: 0.22805874049663544 Train_acc: 93.125008%
Test accuracy of the model: 49.450000%
Test accuracy of the model: 54.083333%
Test accuracy of the mode

Train Epoch: 95/120 Train_Loss: 0.10298073291778564 Train_acc: 97.437500%
Train Epoch: 96/120 Train_Loss: 0.0600007064640522 Train_acc: 97.458336%
Train Epoch: 97/120 Train_Loss: 0.07641533017158508 Train_acc: 97.458336%
Train Epoch: 98/120 Train_Loss: 0.13310836255550385 Train_acc: 97.479172%
Train Epoch: 99/120 Train_Loss: 0.10651518404483795 Train_acc: 97.479172%
Train Epoch: 100/120 Train_Loss: 0.007827896624803543 Train_acc: 97.479172%
Train Epoch: 101/120 Train_Loss: 0.11549994349479675 Train_acc: 97.479172%
Train Epoch: 102/120 Train_Loss: 0.1400485783815384 Train_acc: 97.479172%
Train Epoch: 103/120 Train_Loss: 0.03146546706557274 Train_acc: 97.458336%
Train Epoch: 104/120 Train_Loss: 0.14722348749637604 Train_acc: 97.479172%
Train Epoch: 105/120 Train_Loss: 0.07233971357345581 Train_acc: 97.458336%
Train Epoch: 106/120 Train_Loss: 0.14132075011730194 Train_acc: 97.479172%
Train Epoch: 107/120 Train_Loss: 0.11666727066040039 Train_acc: 97.479172%
Train Epoch: 108/120 Train_Loss

Train Epoch: 79/120 Train_Loss: 0.008741599507629871 Train_acc: 100.000000%
Train Epoch: 80/120 Train_Loss: 0.007390653248876333 Train_acc: 100.000000%
Train Epoch: 81/120 Train_Loss: 0.005672573111951351 Train_acc: 100.000000%
Train Epoch: 82/120 Train_Loss: 0.005380765534937382 Train_acc: 100.000000%
Train Epoch: 83/120 Train_Loss: 0.007733841892331839 Train_acc: 100.000000%
Train Epoch: 84/120 Train_Loss: 0.007352385204285383 Train_acc: 100.000000%
Train Epoch: 85/120 Train_Loss: 0.005427563562989235 Train_acc: 100.000000%
Train Epoch: 86/120 Train_Loss: 0.0061228470876812935 Train_acc: 100.000000%
Train Epoch: 87/120 Train_Loss: 0.007967089302837849 Train_acc: 100.000000%
Train Epoch: 88/120 Train_Loss: 0.004767522681504488 Train_acc: 100.000000%
Train Epoch: 89/120 Train_Loss: 0.006481129210442305 Train_acc: 100.000000%
Train Epoch: 90/120 Train_Loss: 0.006069185212254524 Train_acc: 100.000000%
Train Epoch: 91/120 Train_Loss: 0.0041819289326667786 Train_acc: 100.000000%
Train Epoc

Train Epoch: 64/120 Train_Loss: 0.3214179575443268 Train_acc: 93.112503%
Train Epoch: 65/120 Train_Loss: 0.540097713470459 Train_acc: 93.125008%
Train Epoch: 66/120 Train_Loss: 0.3468312621116638 Train_acc: 93.112503%
Train Epoch: 67/120 Train_Loss: 0.28105807304382324 Train_acc: 93.125008%
Train Epoch: 68/120 Train_Loss: 0.14533020555973053 Train_acc: 93.112503%
Train Epoch: 69/120 Train_Loss: 0.17553028464317322 Train_acc: 93.137505%
Train Epoch: 70/120 Train_Loss: 0.06032047048211098 Train_acc: 93.062508%
Train Epoch: 71/120 Train_Loss: 0.2650884985923767 Train_acc: 93.125008%
Train Epoch: 72/120 Train_Loss: 0.35795357823371887 Train_acc: 93.112503%
Train Epoch: 73/120 Train_Loss: 0.3183058202266693 Train_acc: 93.125008%
Train Epoch: 74/120 Train_Loss: 0.21464219689369202 Train_acc: 93.100006%
Train Epoch: 75/120 Train_Loss: 0.3149738311767578 Train_acc: 93.000008%
Train Epoch: 76/120 Train_Loss: 0.33149364590644836 Train_acc: 93.112503%
Train Epoch: 77/120 Train_Loss: 0.25335362553

...

Train Epoch: 56/120 Train_Loss: 0.005119951441884041 Train_acc: 100.000000%
Train Epoch: 57/120 Train_Loss: 0.005107090808451176 Train_acc: 100.000000%
Train Epoch: 58/120 Train_Loss: 0.006191971246153116 Train_acc: 100.000000%
Train Epoch: 59/120 Train_Loss: 0.005686987191438675 Train_acc: 100.000000%
Train Epoch: 60/120 Train_Loss: 0.006847163196653128 Train_acc: 100.000000%
Train Epoch: 61/120 Train_Loss: 0.005255568772554398 Train_acc: 100.000000%
Train Epoch: 62/120 Train_Loss: 0.007923266850411892 Train_acc: 100.000000%
Train Epoch: 63/120 Train_Loss: 0.0048205251805484295 Train_acc: 100.000000%
Train Epoch: 64/120 Train_Loss: 0.004145822487771511 Train_acc: 100.000000%
Train Epoch: 65/120 Train_Loss: 0.007763535249978304 Train_acc: 100.000000%
Train Epoch: 66/120 Train_Loss: 0.006316609214991331 Train_acc: 100.000000%
Train Epoch: 67/120 Train_Loss: 0.004943982698023319 Train_acc: 100.000000%
Train Epoch: 68/120 Train_Loss: 0.00675106281414628 Train_acc: 100.000000%
Train Epoch:

Train Epoch: 40/120 Train_Loss: 0.3033985495567322 Train_acc: 92.525002%
Train Epoch: 41/120 Train_Loss: 0.19526490569114685 Train_acc: 92.225006%
Train Epoch: 42/120 Train_Loss: 0.3412114381790161 Train_acc: 92.312508%
Train Epoch: 43/120 Train_Loss: 0.37807029485702515 Train_acc: 91.125008%
Train Epoch: 44/120 Train_Loss: 0.28902173042297363 Train_acc: 92.025002%
Train Epoch: 45/120 Train_Loss: 0.16811397671699524 Train_acc: 92.687508%
Train Epoch: 46/120 Train_Loss: 0.2906706631183624 Train_acc: 92.750008%
Train Epoch: 47/120 Train_Loss: 0.3092607855796814 Train_acc: 92.575005%
Train Epoch: 48/120 Train_Loss: 0.31825754046440125 Train_acc: 92.650002%
Train Epoch: 49/120 Train_Loss: 0.6074739694595337 Train_acc: 91.812508%
Train Epoch: 50/120 Train_Loss: 0.38103243708610535 Train_acc: 92.425003%
Train Epoch: 51/120 Train_Loss: 0.23909728229045868 Train_acc: 91.362503%
Train Epoch: 52/120 Train_Loss: 0.20469605922698975 Train_acc: 91.562508%
Train Epoch: 53/120 Train_Loss: 0.231508791

Train Epoch: 26/120 Train_Loss: 0.06287048757076263 Train_acc: 97.416672%
Train Epoch: 27/120 Train_Loss: 0.11169706284999847 Train_acc: 97.333336%
Train Epoch: 28/120 Train_Loss: 0.20786753296852112 Train_acc: 97.312500%
Train Epoch: 29/120 Train_Loss: 0.12564881145954132 Train_acc: 97.375000%
Train Epoch: 30/120 Train_Loss: 0.23232720792293549 Train_acc: 97.437500%
Train Epoch: 31/120 Train_Loss: 0.17532813549041748 Train_acc: 97.395836%
Train Epoch: 32/120 Train_Loss: 0.07854646444320679 Train_acc: 97.354172%
Train Epoch: 33/120 Train_Loss: 0.004752088338136673 Train_acc: 97.354172%
Train Epoch: 34/120 Train_Loss: 0.08391433209180832 Train_acc: 97.375000%
Train Epoch: 35/120 Train_Loss: 0.061439406126737595 Train_acc: 97.333336%
Train Epoch: 36/120 Train_Loss: 0.06727249175310135 Train_acc: 97.416672%
Train Epoch: 37/120 Train_Loss: 0.062173206359148026 Train_acc: 97.312500%
Train Epoch: 38/120 Train_Loss: 0.07241632044315338 Train_acc: 97.416672%
Train Epoch: 39/120 Train_Loss: 0.1

Train Epoch: 12/120 Train_Loss: 0.13950981199741364 Train_acc: 97.000000%
Train Epoch: 13/120 Train_Loss: 0.0929543524980545 Train_acc: 96.312500%
Train Epoch: 14/120 Train_Loss: 0.11837749183177948 Train_acc: 96.875000%
Train Epoch: 15/120 Train_Loss: 0.1618555635213852 Train_acc: 97.937500%
Train Epoch: 16/120 Train_Loss: 0.06638455390930176 Train_acc: 98.187500%
Train Epoch: 17/120 Train_Loss: 0.04060748219490051 Train_acc: 99.125000%
Train Epoch: 18/120 Train_Loss: 0.13417908549308777 Train_acc: 97.625000%
Train Epoch: 19/120 Train_Loss: 0.05921325832605362 Train_acc: 98.187500%
Train Epoch: 20/120 Train_Loss: 0.02617257833480835 Train_acc: 99.437500%
Train Epoch: 21/120 Train_Loss: 0.030870145186781883 Train_acc: 99.812500%
Train Epoch: 22/120 Train_Loss: 0.010791826993227005 Train_acc: 100.000000%
Train Epoch: 23/120 Train_Loss: 0.0022530427668243647 Train_acc: 100.000000%
Train Epoch: 24/120 Train_Loss: 0.0026689444202929735 Train_acc: 100.000000%
Train Epoch: 25/120 Train_Loss:

Train Epoch: 120/120 Train_Loss: 0.005320426542311907 Train_acc: 100.000000%
Test accuracy of the model: 47.550000%
Test accuracy of the model: 61.000000%
Test accuracy of the model: 66.000000%
Training Time: 207.88 seconds


In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()